In [1]:
from utils import load_cnf, cnf_to_bqm, get_ancilla_index

In [2]:
def load_file(folder = 'sat', _variables=140):
    path = f"./{folder}/sgen1-{folder}-{_variables}-100.cnf"
    num_variables, num_clauses, variables, clauses = load_cnf(path)
    return num_variables, num_clauses, variables, clauses

In [3]:
def get_key_with_value(d, val):
    for (key, value) in d.items():
        if d[key] == val:
            return key
        
    return -1

def mirror_variable(variables, clauses) -> (int, int):
    # find variables that repeats the most
    counts = dict()
    for clause in clauses:
        for var in clause:
            abs_var = abs(var)
            if abs_var not in counts:
                counts[abs_var] = 0
            counts[abs_var] += 1
    
    max_count = max(counts.values())
    key = get_key_with_value(counts, max_count)
    assert(key != -1)
    assert(counts[key] == max_count)
    new_variable = len(variables) + 1 
    assert(new_variable not in variables)
    variables.add(new_variable)
    replacements = max_count//2
    if replacements == 0:
        return key, new_variable
    for clause in clauses:
        for i in range(len(clause)):
            if abs(clause[i]) == key:
                if clause[i] < 0:
                    clause[i] = -new_variable
                else:
                    clause[i] = new_variable
                replacements -= 1
                if replacements == 0:
                    return key, new_variable
    return -1, -1

In [12]:
num_variables, num_clauses, variables, clauses = load_file( _variables=180)

In [5]:
num_variables_to_mirror = 1
variables_to_mirror = []
for i in range(num_variables_to_mirror):
    variables_to_mirror.append(mirror_variable(variables, clauses))

In [13]:
bqm, or_result_vars, clauses_qubits = cnf_to_bqm(variables, clauses)

In [14]:
len(bqm.variables)

396

In [4]:
def update_bqm_mirror_variable(original, mirror):
    global bqm
    bqm.add_variable(original, 2)
    bqm.add_variable(mirror, 2)
    bqm.add_interaction(original, mirror, -4)

In [8]:
for vtm in variables_to_mirror:
    update_bqm_mirrow_variable(vtm[0], vtm[1])

In [5]:
from minorminer import find_embedding
from dwave.system import DWaveSampler, FixedEmbeddingComposite
qpu_pegasus = DWaveSampler(solver={'topology__type': 'pegasus'})

In [6]:
embedding = find_embedding(bqm.quadratic.keys(), qpu_pegasus.edgelist, random_seed=1)

def get_chain_lengths(_embedding):
    lengths = []
    biases = []
    for (key, value) in _embedding.items():
        lengths.append(len(value))
        biases.append(bqm.linear[key])
    return lengths, biases

import seaborn as sns
sns.histplot(get_chain_lengths(embedding)[0])

NameError: name 'bqm' is not defined

In [6]:
def count_qubits_used(embedding):
    physical_vars = set()
    for (logic_var, chain) in embedding.items():
        for pv in chain:
            physical_vars.add(pv)
        
    return len(physical_vars)
#count_qubits_used(embedding)

-----------------------

In [15]:
mirrored_variables = []
total_used_qubits = []
for num_variables_to_mirror in range( 180):
    
    num_variables, num_clauses, variables, clauses = load_file( _variables=180)
    variables_to_mirror = []
    for i in range(num_variables_to_mirror):
        variables_to_mirror.append(mirror_variable(variables, clauses))
    bqm, or_result_vars, clauses_qubits = cnf_to_bqm(variables, clauses)
    assert(len(bqm.variables) == 396 + num_variables_to_mirror)
    for vtm in variables_to_mirror:
        update_bqm_mirror_variable(vtm[0], vtm[1])
    embedding = find_embedding(bqm.quadratic.keys(), qpu_pegasus.edgelist, random_seed=1)
    mirrored_variables.append(num_variables_to_mirror)
    total_used_qubits.append(count_qubits_used(embedding))
    print(num_variables_to_mirror, count_qubits_used(embedding))

0 2147
1 1843
2 2181
3 2175
4 2050
5 2154
6 2072
7 2393
8 2075
9 2172
10 2148
11 2256
12 2256
13 2404
14 2245
15 2452
16 2286
17 2118
18 2160
19 2108
20 2051
21 2178
22 1986
23 2295
24 2385
25 2241
26 2155
27 2378
28 2287
29 2091
30 2207
31 2064
32 2082
33 2336
34 2175
35 2257
36 2156
37 2479
38 2054
39 2216
40 2434
41 2121
42 2047
43 2177
44 2245
45 2142
46 2210
47 2106
48 2323
49 2069
50 2299
51 2558
52 2290
53 2580
54 2095
55 2273
56 2120
57 2287
58 2420
59 2477
60 2280
61 2307
62 2398
63 2363
64 2286
65 2278
66 2289
67 2261
68 1972
69 2276
70 2338
71 2215
72 2264
73 2214
74 2308
75 2432
76 2390
77 2500
78 2182
79 2534
80 2310
81 2485
82 2320
83 2466
84 2447
85 2534
86 2533
87 2558
88 2691
89 2258
90 2298
91 2462
92 2445
93 2380
94 2224
95 2301
96 2502
97 2244
98 2219
99 2439
100 2411
101 2356
102 2562
103 2644
104 2384
105 2610
106 2542
107 2348
108 2414
109 2500
110 2393
111 2249
112 2474
113 2482
114 2399
115 2358
116 2310
117 2514
118 2498
119 2533
120 2407
121 2483
122 2764
123